In [1]:
%load_ext lab_black

In [2]:
import json
import requests
import yaml

url = "https://ckan.opendata.swiss/api/3/action/package_show?id=haushaltsaquivalenz-einkommen-steuerpflichtiger-naturlicher-personen-nach-haushaltstyp-stadtqua"

resp = requests.get(url)
resp.content[:500]

b'{"help": "https://ckan.opendata.swiss/api/3/action/help_show?name=package_show", "success": true, "result": {"license_title": null, "maintainer": "Open Data Z\\u00fcrich", "issued": "2022-11-01T00:00:00", "title_for_slug": "haushaltsaquivalenz-einkommen-steuerpflichtiger-naturlicher-personen-nach-haushaltstyp-stadtqua", "qualified_relations": [], "private": false, "maintainer_email": "opendata@zuerich.ch", "num_tags": 7, "contact_points": [{"email": "opendata@zuerich.ch", "name": "Open Data Z\\u00'

In [3]:
resp.status_code

200

In [4]:
resp.encoding

'utf-8'

In [5]:
json_household_income = resp.json()
household_income_url = json_household_income["result"]["resources"][0]["url"]

In [6]:
import pandas as pd

zurich_household_income = pd.read_csv(household_income_url)

In [7]:
zurich_household_income_ytd = zurich_household_income.loc[
    zurich_household_income["StichtagDatJahr"]
    == zurich_household_income["StichtagDatJahr"].max()
]
zurich_household_income_ytd = zurich_household_income_ytd.dropna()
zurich_household_income_ytd.head(10)

# Tax year (technically: key date date year)	Year for which the tax was claimed (integer).
# City quarter (sort) (technically: QuarSort)	ID for the city district at the current residential address (integer).
# City quarter (code) (technical: QuarCd)	ID for the city district at the current residential address (string).
# City Quarter (Lang) (technically: QuarLang)	City district name of the current residential address (string).
# Household form sort (technically: HHtypSort)	Household form sorting code (integer).
# Household type code (technical: HHtypCd)	Household form sorting code (integer).
# Household type (technically: HHtypLang)	Household form description (string).
# Median value (technical: aequEK_p50)	Median value of the household equivalent income of natural persons in Zurich. In thousand francs (integers).
# P25 value (technical: aequEK_p25)	25% quantile value of the household equivalent income of natural persons in Zurich. In thousand francs (integers).
# P75 value (technical: aequEK_p75)	75% quantile value of the household equivalent income of natural persons in Zurich. In thousand francs (integers).

# https://data.stadt-zuerich.ch/dataset/fd_median_hhaequeinkommen_quartier_od100a

,StichtagDatJahr,QuarSort,QuarCd,QuarLang,HHtypSort,HHtypCd,HHtypLang,aequEK_p50,aequEK_p25,aequEK_p75
2676,2020,11,11,Rathaus,7,1EH,Ein-Elternhaushalt,44.93,20.31,63.23
2677,2020,11,11,Rathaus,8,EHK,Ehepaar mit Kind(ern),64.29,41.84,92.92
2678,2020,11,11,Rathaus,5,EH,Ehepaar ohne Kind(er),75.73,45.53,130.67
2679,2020,11,11,Rathaus,4,2P,Zweipersonenhaushalt,81.87,50.73,110.13
2680,2020,11,11,Rathaus,1,1P,Einpersonenhaushalt,58.30,28.60,100.15
2689,2020,12,12,Hochschulen,1,1P,Einpersonenhaushalt,58.40,29.30,107.50
2696,2020,13,13,Lindenhof,8,EHK,Ehepaar mit Kind(ern),44.12,29.29,83.22
2697,2020,13,13,Lindenhof,5,EH,Ehepaar ohne Kind(er),76.27,51.00,121.27
2698,2020,13,13,Lindenhof,4,2P,Zweipersonenhaushalt,90.07,59.80,139.47
2699,2020,13,13,Lindenhof,1,1P,Einpersonenhaushalt,67.10,33.70,107.90


In [8]:
zurich_household_income_ytd_median = (
    zurich_household_income_ytd.groupby(["QuarLang", "HHtypLang"])
    .agg("median")
    .reset_index()
)

household_dict = {
    "Ehepaar mit Kind(ern)": "Married couple with child(ren)",
    "Ehepaar ohne Kind(er)": "Married couple without child(ren)",
    "Ein-Elternhaushalt": "One-parent household",
    "Einpersonenhaushalt": "One-person household",
    "Paar mit Kind(ern)": "Couple with child(ren)",
    "Patchwork-Haushalt": "Patchwork household",
    "Wohngemeinschaft mit Kind(ern)": "Shared apartment with child(ren)",
    "Wohngemeinschaft ohne Kind(er)": "Shared apartment without child(ren)",
    "Zweipersonenhaushalt": "Two-person household",
    "Eingetragenes Paar ohne Kind(er)": "Registered couple without child(ren)",
}

zurich_household_income_ytd_median = zurich_household_income_ytd_median.replace(
    {"HHtypLang": household_dict}
)

zurich_household_income_ytd_median.to_csv("zurich_household_income_ytd_median.csv")
zurich_household_income_ytd_median

,QuarLang,HHtypLang,StichtagDatJahr,QuarSort,QuarCd,HHtypSort,aequEK_p50,aequEK_p25,aequEK_p75
0,Affoltern,Married couple with child(ren),2020.0,111.0,111.0,8.0,39.24,24.52,56.14
1,Affoltern,Married couple without child(ren),2020.0,111.0,111.0,5.0,52.00,37.27,70.93
2,Affoltern,One-parent household,2020.0,111.0,111.0,7.0,30.00,15.07,46.53
3,Affoltern,One-person household,2020.0,111.0,111.0,1.0,45.20,24.90,65.20
4,Affoltern,Couple with child(ren),2020.0,111.0,111.0,10.0,42.22,32.33,59.33
...,...,...,...,...,...,...,...,...,...
266,Wollishofen,One-person household,2020.0,21.0,21.0,1.0,53.55,32.70,82.15
267,Wollishofen,Couple with child(ren),2020.0,21.0,21.0,10.0,47.28,32.67,69.08
268,Wollishofen,Shared apartment with child(ren),2020.0,21.0,21.0,13.0,53.05,32.33,82.19
269,Wollishofen,Shared apartment without child(ren),2020.0,21.0,21.0,12.0,66.03,44.44,101.45


In [9]:
url2 = "https://ckan.opendata.swiss/api/3/action/package_show?id=privathaushalte-nach-haushaltsform-und-stadtquartier-seit-20131"

resp = requests.get(url2)
resp.content[:500]

b'{"help": "https://ckan.opendata.swiss/api/3/action/help_show?name=package_show", "success": true, "result": {"license_title": null, "maintainer": "Open Data Z\\u00fcrich", "issued": "2016-11-24T00:00:00", "title_for_slug": "privathaushalte-nach-haushaltsform-und-stadtquartier-seit-20131", "qualified_relations": [], "private": false, "maintainer_email": "opendata@zuerich.ch", "num_tags": 12, "contact_points": [{"email": "opendata@zuerich.ch", "name": "Open Data Z\\u00fcrich"}], "keywords": {"fr": ['

In [10]:
resp.status_code

200

In [11]:
resp.encoding

'utf-8'

In [12]:
json_household_form = resp.json()
household_form_url = json_household_form["result"]["resources"][0]["url"]

In [13]:
zurich_household_form = pd.read_csv(household_form_url)

In [20]:
zurich_household_recentyear = zurich_household_income["StichtagDatJahr"].max()
zurich_household_form_ytd = zurich_household_form.loc[
    zurich_household_form["StichtagDatJahr"] == zurich_household_recentyear
]
zurich_household_form_ytd = zurich_household_form_ytd.dropna()
zurich_household_form_ytd.head(10)

# Year (technically: key date date year)	Year-end inventory (numeric)
# City quarter sorting code (technically: QuarSort)	Official ID of the statistical city district (numeric)
# City district description (technically: QuarLang)	Name of the statistical district (string)
# City district sorting code (technically: KreisSort)	ID and sorting variable for the city district at the current residential address (Numeric)
# City district description (technically: KreisLang)	City district name of the current residential address (string)
# Household type sorting code (technically: HHtypSort)	Household type sorting code (numeric)
# Household type description (technical: HHtypLang)	Household type description (string)
# Number of private households (technically: AnzHH)	Number of private households (numerical)
# Economic resident population (technically: AnzBestWe)	Economic Resident Population in Private Households (Numerical)

# https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsform_quartier_seit2013_od3805

,StichtagDatJahr,QuarSort,QuarLang,KreisSort,KreisLang,HHtypSort,HHtypLang,AnzHH,AnzBestWir
2755,2020,11,Rathaus,1,Kreis 1,1,Einpersonenhaushalt,1170,1170
2756,2020,11,Rathaus,1,Kreis 1,4,Zweipersonenhaushalt,283,566
2757,2020,11,Rathaus,1,Kreis 1,5,Ehepaar ohne Kind(er),170,340
2758,2020,11,Rathaus,1,Kreis 1,6,Eingetragenes Paar ohne Kind(er),13,26
2759,2020,11,Rathaus,1,Kreis 1,7,Ein-Elternhaushalt,65,145
2760,2020,11,Rathaus,1,Kreis 1,8,Ehepaar mit Kind(ern),109,408
2761,2020,11,Rathaus,1,Kreis 1,9,Eingetragenes Paar mit Kind(ern),1,3
2762,2020,11,Rathaus,1,Kreis 1,10,Paar mit Kind(ern),15,59
2763,2020,11,Rathaus,1,Kreis 1,11,Patchwork-Haushalt,5,17
2764,2020,11,Rathaus,1,Kreis 1,12,Wohngemeinschaft ohne Kind(er),82,293


In [21]:
zurich_household_form_ytd_sum = (
    zurich_household_form_ytd.groupby(["QuarLang", "HHtypLang"])
    .agg("sum")
    .reset_index()
)

household_dict = {
    "Ehepaar mit Kind(ern)": "Married couple with child(ren)",
    "Ehepaar ohne Kind(er)": "Married couple without child(ren)",
    "Ein-Elternhaushalt": "One-parent household",
    "Einpersonenhaushalt": "One-person household",
    "Paar mit Kind(ern)": "Couple with child(ren)",
    "Patchwork-Haushalt": "Patchwork household",
    "Wohngemeinschaft mit Kind(ern)": "Shared apartment with child(ren)",
    "Wohngemeinschaft ohne Kind(er)": "Shared apartment without child(ren)",
    "Zweipersonenhaushalt": "Two-person household",
    "Eingetragenes Paar ohne Kind(er)": "Registered couple without child(ren)",
}

zurich_household_form_ytd_sum = zurich_household_form_ytd_sum.replace(
    {"HHtypLang": household_dict}
)

zurich_household_form_ytd_sum

,QuarLang,HHtypLang,StichtagDatJahr,QuarSort,KreisSort,HHtypSort,AnzHH,AnzBestWir
0,Affoltern,Married couple with child(ren),2020,111,11,8,2440,9326
1,Affoltern,Married couple without child(ren),2020,111,11,5,1658,3316
2,Affoltern,One-parent household,2020,111,11,7,753,1883
3,Affoltern,Eingetragenes Paar mit Kind(ern),2020,111,11,9,4,14
4,Affoltern,Registered couple without child(ren),2020,111,11,6,28,56
...,...,...,...,...,...,...,...,...
394,Wollishofen,Couple with child(ren),2020,21,2,10,315,1154
395,Wollishofen,Patchwork household,2020,21,2,11,44,146
396,Wollishofen,Shared apartment with child(ren),2020,21,2,13,122,589
397,Wollishofen,Shared apartment without child(ren),2020,21,2,12,257,865


In [77]:
quartiers_household_count = (
    zurich_household_form_ytd_sum.groupby(["QuarLang"]).agg("sum").reset_index()
)

quartiers_household_count.drop(
    ["StichtagDatJahr", "QuarSort", "KreisSort", "HHtypSort"],
    inplace=True,
    axis=1,
    errors="ignore",
)
quartiers_household_count.to_csv("zurich_quartiers_household_count.csv")
quartiers_household_count

,QuarLang,AnzHH,AnzBestWir
0,Affoltern,11549,25658
1,Albisrieden,10877,21803
2,Alt-Wiedikon,8876,17012
3,Altstetten,16869,33331
4,City,310,702
5,Enge,4587,9299
6,Escher Wyss,2998,6151
7,Fluntern,3858,8277
8,Friesenberg,4307,10671
9,Gewerbeschule,4939,9077


In [29]:
zurich_household_merge_summary = pd.merge(
    zurich_household_income_ytd_median,
    zurich_household_form_ytd_sum,
    how="left",
    on=["QuarLang", "HHtypLang"],
)
zurich_household_merge_summary.drop(
    ["HHtypSort_x", "StichtagDatJahr_y", "QuarSort_y", "KreisSort", "HHtypSort_y"],
    inplace=True,
    axis=1,
    errors="ignore",
)
zurich_household_merge_summary

,QuarLang,HHtypLang,StichtagDatJahr_x,QuarSort_x,QuarCd,aequEK_p50,aequEK_p25,aequEK_p75,AnzHH,AnzBestWir
0,Affoltern,Married couple with child(ren),2020.0,111.0,111.0,39.24,24.52,56.14,2440,9326
1,Affoltern,Married couple without child(ren),2020.0,111.0,111.0,52.00,37.27,70.93,1658,3316
2,Affoltern,One-parent household,2020.0,111.0,111.0,30.00,15.07,46.53,753,1883
3,Affoltern,One-person household,2020.0,111.0,111.0,45.20,24.90,65.20,4233,4233
4,Affoltern,Couple with child(ren),2020.0,111.0,111.0,42.22,32.33,59.33,265,962
...,...,...,...,...,...,...,...,...,...,...
266,Wollishofen,One-person household,2020.0,21.0,21.0,53.55,32.70,82.15,4102,4102
267,Wollishofen,Couple with child(ren),2020.0,21.0,21.0,47.28,32.67,69.08,315,1154
268,Wollishofen,Shared apartment with child(ren),2020.0,21.0,21.0,53.05,32.33,82.19,122,589
269,Wollishofen,Shared apartment without child(ren),2020.0,21.0,21.0,66.03,44.44,101.45,257,865


In [98]:
quartiers = zurich_household_merge_summary["QuarLang"].unique().tolist()

zurich_household_income_quartier = pd.DataFrame(columns=["QuarLang", "HHincome_median"])
zurich_household_income_quartier["QuarLang"] = quartiers

for quartier in quartiers:
    df_quartier = zurich_household_merge_summary[
        zurich_household_merge_summary["QuarLang"] == quartier
    ]
    df_quartier.sort_values("aequEK_p50", inplace=True)
    cumsum = df_quartier.AnzHH.cumsum()
    cutoff = df_quartier.AnzHH.sum() / 2.0
    median = df_quartier.aequEK_p50[cumsum >= cutoff].iloc[0]
    zurich_household_income_quartier.at[
        quartiers.index(quartier), "HHincome_median"
    ] = median

zurich_household_income_quartier.to_csv("zurich_household_income_quartier.csv")
zurich_household_income_quartier

/var/folders/bx/zk0r2vl519d9zswyfq9yq3700000gn/T/ipykernel_98743/2521810564.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quartier.sort_values("aequEK_p50", inplace=True)


,QuarLang,HHincome_median
0,Affoltern,45.2
1,Albisrieden,51.0
2,Alt-Wiedikon,61.2
3,Altstetten,49.6
4,City,70.7
5,Enge,75.0
6,Escher Wyss,78.7
7,Fluntern,86.08
8,Friesenberg,44.9
9,Gewerbeschule,52.7
